In [1]:
# Import python packages
import os
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [2]:
# Can uncomment to install R packages as needed
#utils = importr('utils')
#utils.install_packages('nflreadr')
#utils.install_packages('ffscrapr')

In [3]:
# Import R packages
ffscrapr = importr('ffscrapr')

In [4]:
# Config for the calculation
mfl_id = 60206  #MFL id for league
current_season = 2024
past_seasons = [2023,2022,2021,2020]  #Seasons to collect data for
current_week = 0
max_week = 17  #Maximum week number to include in calculation
min_games = 8
save_label = 'Analytics_Dynasty_League_2024'  #Where to save results

load_data = True

In [5]:
# Map from teams to conferences
conf_dic = {
    'Arizona Cardinals':'NFC',
    'Atlanta Falcons':'NFC',
    'Baltimore Ravens':'AFC',
    'Buffalo Bills':'AFC',
    'Carolina Panthers':'NFC',
    'Chicago Bears':'NFC',
    'Cincinnati Bengals':'AFC',
    'Cleveland Browns':'AFC',
    'Dallas Cowboys':'NFC',
    'Denver Broncos':'AFC',
    'Detroit Lions':'NFC',
    'Green Bay Packers':'NFC',
    'Houston Texans':'AFC',
    'Indianapolis Colts':'AFC',
    'Jacksonville Jaguars':'AFC',
    'Kansas City Chiefs':'AFC',
    'Las Vegas Raiders':'AFC',
    'Los Angeles Chargers':'AFC',
    'Los Angeles Rams':'NFC',
    'Miami Dolphins':'AFC',
    'Minnesota Vikings':'NFC',
    'New England Patriots':'AFC',
    'New Orleans Saints':'NFC',
    'New York Giants':'NFC',
    'New York Jets':'AFC',
    'Philadelphia Eagles':'NFC',
    'Pittsburgh Steelers':'AFC',
    'San Francisco 49ers':'NFC',
    'Seattle Seahawks':'NFC',
    'Tampa Bay Buccaneers':'NFC',
    'Tennessee Titans':'AFC',
    'Washington Commanders':'NFC',
    'Washington Football Team':'NFC',
}

In [6]:
# Map from positions to rank floors
rank_floor_dic = {
    'CB':36, 
    'DE':40, 
    'DT':36, 
    'LB':40, 
    'PK':16, 
    'PN':16, 
    'QB':16, 
    'RB':28, 
    'S':40, 
    'TE':16, 
    'WR':52,
}

In [7]:
# Check whether or not to load roster and playerscore data from disk
if load_data and os.path.isfile(f'{save_label}_Rosters.csv') and os.path.isfile(f'{save_label}_PlayerScores.csv'):
    # Will load roster and playerscore data from disk
    print('Loading roster and playerscore data from disk')
    
    rosters_df = pd.read_csv(f'{save_label}_Rosters.csv')
    playerscores_df = pd.read_csv(f'{save_label}_PlayerScores.csv')

else:
    # Will scrape roster  and playerscore data from MFL
    print('Scraping roster and playerscore data from MFL')
    
    # Dict to store data for each season
    rosters_df = {}
    playerscores_df = {}

    # !!! NEED TO SCRAPE DATA FOR CURRENT SEASON TOO, ADD WHEN APPROPRIATE !!!
    for s in past_seasons:
        # Scrape player positions from MFL and convert to pandas df
        mfl = ffscrapr.mfl_connect(season=s, league_id=mfl_id, rate_limit_number=1, rate_limit_seconds=6)
        
        rosters_df_r = ffscrapr.ff_rosters(mfl)
        playerscores_df_r = ffscrapr.ff_playerscores(mfl, season=s, week=[i+1 for i in range(max_week)])
        
        with (ro.default_converter + pandas2ri.converter).context():
            rosters_df[s] = ro.conversion.get_conversion().rpy2py(rosters_df_r)
            playerscores_df[s] = ro.conversion.get_conversion().rpy2py(playerscores_df_r)
    
        # Cleanup some of the data
        rosters_df[s]['season'] = s
        rosters_df[s]['player_id'] = rosters_df[s]['player_id'].astype(int)
    
        playerscores_df[s]['season'] = playerscores_df[s]['season'].astype(int)
        playerscores_df[s]['week'] = playerscores_df[s]['week'].astype(int)
        playerscores_df[s]['player_id'] = playerscores_df[s]['player_id'].astype(int)
        playerscores_df[s]['points'] = playerscores_df[s]['points'].astype(float)
        playerscores_df[s] = playerscores_df[s].drop('is_available', axis=1)
    
    # Merge seasons to single df
    rosters_df = pd.concat(rosters_df, ignore_index=True)
    playerscores_df = pd.concat(playerscores_df, ignore_index=True)
    
    # Sort by for cleaner display
    rosters_df = rosters_df.set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True]).reset_index()
    playerscores_df = playerscores_df.set_index(['player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True]).reset_index()
    
    # Save a copy of the roster and playerscore dfs
    rosters_df.to_csv(f'{save_label}_Rosters.csv', index=False)
    playerscores_df.to_csv(f'{save_label}_PlayerScores.csv', index=False)

Loading roster and playerscore data from disk


In [8]:
rosters_df

,player_id,season,franchise_id,franchise_name,player_name,pos,team,age,salary,contract_years,contractInfo,roster_status,draft_year,draft_round
0,4925,2020,3,Philadelphia Eagles,"Brees, Drew",QB,NOS,45.2,22.36,1.0,2020 NEFT,ROSTER,2001,2
1,4925,2020,24,Pittsburgh Steelers,"Brees, Drew",QB,NOS,45.2,15.50,2.0,2020 UFA,ROSTER,2001,2
2,5848,2020,14,Los Angeles Rams,"Brady, Tom",QB,TBB,46.6,22.36,1.0,2020 NEFT,ROSTER,2000,6
3,5848,2020,32,Los Angeles Chargers,"Brady, Tom",QB,TBB,46.6,22.70,1.0,2020 UFA,ROSTER,2000,6
4,5848,2021,4,Washington Football Team,"Brady, Tom",QB,TBB,46.6,15.00,2.0,2021 UFA,ROSTER,2000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978,16441,2023,17,Buffalo Bills,"DeVito, Tommy",QB,NYG,25.6,0.90,1.0,2023 UFA,ROSTER,2023,NA_character_
5979,16444,2023,17,Buffalo Bills,"Grupe, Blake",PK,NOS,25.3,0.90,1.0,2023 UFA,ROSTER,2023,NA_character_
5980,16457,2023,2,New York Giants,"Izien, Christian",CB,TBB,23.8,1.50,1.0,2023 UFA,ROSTER,2023,NA_character_
5981,16458,2023,17,Buffalo Bills,"Hedley, Lou",PN,NOS,30.7,0.90,1.0,2023 UFA,ROSTER,2023,NA_character_


In [9]:
playerscores_df

,player_id,season,week,player_name,pos,team,points
0,4925,2020,1,"Brees, Drew",QB,NOS,19.7
1,4925,2020,2,"Brees, Drew",QB,NOS,39.9
2,4925,2020,3,"Brees, Drew",QB,NOS,44.5
3,4925,2020,4,"Brees, Drew",QB,NOS,31.5
4,4925,2020,5,"Brees, Drew",QB,NOS,36.8
...,...,...,...,...,...,...,...
77650,16560,2023,16,"Maddox-Williams, Tyreek",LB,ARI,0.0
77651,16560,2023,17,"Maddox-Williams, Tyreek",LB,ARI,0.0
77652,16562,2023,16,"Williams, Jaylin",CB,MIN,-1.0
77653,16562,2023,17,"Williams, Jaylin",CB,MIN,0.0


In [10]:
# Check whether or not to load contract data from disk
if load_data and os.path.isfile(f'{save_label}_Contracts.csv'):
    # Will load contract data from disk
    print('Loading contract data from disk')
    
    contracts_df = pd.read_csv(f'{save_label}_Contracts.csv')

else:
    # Will calculate contract data from roster and playerscore data
    print('Calculating contract data from roster and playerscore data')

    # Determine if player participated in enough games for season to be considered robus
    playerscores_df = playerscores_df.set_index(['player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])
    playerscores_df['num_games'] = playerscores_df['points'].groupby(['player_id','season']).count()
    playerscores_df['is_robust'] = playerscores_df['num_games'] >= min_games
    
    # Aggregate and rank qualifying player scores by position
    contracts_df = playerscores_df.groupby(['player_id','season']).head(1).droplevel(2).drop('points', axis=1)
    contracts_df['tot_pts'] = playerscores_df['points'].groupby(['player_id','season']).sum()
    contracts_df['avg_pts'] = playerscores_df['points'].groupby(['player_id','season']).mean()

    playerscores_df = playerscores_df.reset_index()
    contracts_df = contracts_df.reset_index().set_index(['pos','season']).sort_index(level=[0,1],ascending=[True,True])
    
    contracts_df['tot_pts_rank'] = contracts_df['tot_pts'].groupby(['pos','season']).rank(method='average',ascending=False).values
    contracts_df['avg_pts_rank'] = contracts_df['avg_pts'].groupby(['pos','season']).rank(method='average',ascending=False).values
    contracts_df['floor_pts_rank'] = contracts_df.index.get_level_values(0).map(lambda x: rank_floor_dic[x])
    
    contracts_df = contracts_df.reset_index().set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True])
    
    # Merge in aggregated playerscores to roster_df
    contracts_df = rosters_df.set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True]).merge(contracts_df[['tot_pts','tot_pts_rank','avg_pts','avg_pts_rank','floor_pts_rank','num_games','is_robust']], how='left', left_index=True, right_index=True)
    contracts_df['conf'] = contracts_df['franchise_name'].map(lambda x: conf_dic[x])
    
    # Rank the salaries by position
    contracts_df = contracts_df.reset_index().set_index(['pos','season']).sort_index(level=[0,1],ascending=[True,True])
    contracts_df['salary_rank'] = contracts_df['salary'].groupby(['pos','season']).rank(method='first',ascending=False).values
    contracts_df = contracts_df.reset_index()
    
    # Sort by for cleaner display
    contracts_df = contracts_df.set_index(['pos','season','salary_rank','conf']).sort_index(level=[0,1,2,3],ascending=[True,True,True,True]).reset_index()
    
    # Save a copy of the contract df
    contracts_df.to_csv(f'{save_label}_Contracts.csv', index=False)

Loading contract data from disk


In [11]:
contracts_df

,pos,season,salary_rank,conf,player_id,franchise_id,franchise_name,player_name,team,age,...,roster_status,draft_year,draft_round,tot_pts,tot_pts_rank,avg_pts,avg_pts_rank,floor_pts_rank,num_games,is_robust
0,CB,2020,1.0,AFC,12245,24,Pittsburgh Steelers,"Peters, Marcus",BAL,31.2,...,ROSTER,2015,1,120.9,20.0,9.300000,15.0,36.0,13.0,True
1,CB,2020,2.0,NFC,12245,3,Philadelphia Eagles,"Peters, Marcus",BAL,31.2,...,ROSTER,2015,1,120.9,20.0,9.300000,15.0,36.0,13.0,True
2,CB,2020,3.0,NFC,13249,9,Atlanta Falcons,"White, Tre'Davious",BUF,29.1,...,ROSTER,2017,1,130.5,13.0,8.700000,22.0,36.0,15.0,True
3,CB,2020,4.0,AFC,10789,22,Cincinnati Bengals,"Gilmore, Stephon",NEP,33.5,...,ROSTER,2012,1,54.7,100.0,4.972727,103.0,36.0,11.0,True
4,CB,2020,5.0,AFC,11764,29,Denver Broncos,"Fuller, Kyle",CHI,32.1,...,ROSTER,2014,1,91.4,43.0,5.712500,82.0,36.0,16.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978,WR,2023,237.0,AFC,16199,27,Jacksonville Jaguars,"Palmer, Trey",TBB,22.9,...,ROSTER,2023,6,37.4,101.0,2.337500,117.0,52.0,16.0,True
5979,WR,2023,238.0,AFC,16201,20,New York Jets,"Hutchinson, Xavier",HOU,23.8,...,ROSTER,2023,6,14.6,146.0,0.973333,158.0,52.0,15.0,True
5980,WR,2023,239.0,NFC,16207,6,Detroit Lions,"Iosivas, Andrei",CIN,24.4,...,ROSTER,2023,6,18.0,136.0,1.200000,148.0,52.0,15.0,True
5981,WR,2023,240.0,AFC,16299,29,Denver Broncos,"Davis, Derius",LAC,23.5,...,ROSTER,2023,4,24.4,124.0,1.525000,137.0,52.0,16.0,True


In [12]:
###
def PtsRankToSalary(pts_rank, salaries):
    # Map points rank to corresponding salary
    if np.isnan(pts_rank):
        calculated_salary = np.nan

    elif pts_rank == 1:
        calculated_salary = salaries[0] + salaries[1] - 0.5*salaries[2] - 0.5*salaries[3]
        
    else:
        sal_eval_rank_high = int(np.round(2*pts_rank - 3))
        sal_eval_rank_low = int(np.round(2*pts_rank - 2))
    
        sal_eval_high = salaries[sal_eval_rank_high - 1]
        sal_eval_low = salaries[sal_eval_rank_low - 1]
        
        calculated_salary = 0.5*sal_eval_high + 0.5*sal_eval_low

    return calculated_salary

###
def CalculateBaselineEYS(df, player_id, player_conference, current_season, current_week, min_salary=1.7):
    # Function that calculates the baseline extended years salary (EYS)
    
    # Some logic to make sure pre-season vs in-season calculation is handled correctly
    query_season = current_season-1 if (current_week==0) else current_season
    salary_multiplier = 1.1 if (current_week==0) else 1.

    # Downselect data based on player_id and salary year
    player_df = df[ (df['player_id'] == player_id) & (df['conf'] == player_conference) ]
    salary_df = salary_multiplier * df[ (df['season'] == query_season) ][['pos','salary_rank','salary']].set_index(['pos','salary_rank']).sort_index(level=[0,1],ascending=[True,True])['salary']

    #print(player_df['player_name'].values[0])
    
    # Initialize list to store calculated salaries
    calculated_salaries = [min_salary]

    # Find salary, player ranks, and calculated salary for current season
    tmp_df = player_df[ player_df['season'] == query_season ]

    #display(tmp_df)
    
    current_salary = salary_multiplier * tmp_df['salary'].values[0]
    current_contract_years = tmp_df['contract_years'].values[0]
    calculated_salaries += [current_salary]

    for tmp_pos,x,y,z in zip(tmp_df['pos'].values, tmp_df['tot_pts_rank'].values, tmp_df['avg_pts_rank'].values, tmp_df['floor_pts_rank'].values):
        min_pts_ranks = min(tmp_df['tot_pts_rank'].min(), tmp_df['avg_pts_rank'].min(), tmp_df['floor_pts_rank'].min())
        calculated_salaries += [ PtsRankToSalary( min(x,y,z), salary_df.loc[tmp_pos].values ) ]

        #print(tmp_pos)
        #print(calculated_salaries)
    
    # Find player ranks and calculated salaries for previous robust seasons
    tmp_df = player_df[ (player_df['season'] < current_season) & (player_df['is_robust']) ].tail(2)

    #display(tmp_df)
    
    for tmp_pos,x,y,z in zip(tmp_df['pos'].values, tmp_df['tot_pts_rank'].values, tmp_df['avg_pts_rank'].values, tmp_df['floor_pts_rank'].values):
        min_pts_ranks = min(tmp_df['tot_pts_rank'].min(), tmp_df['avg_pts_rank'].min(), tmp_df['floor_pts_rank'].min())
        calculated_salaries += [ PtsRankToSalary( min(x,y,z), salary_df.loc[tmp_pos].values ) ]

        #print(tmp_pos)
        #print(calculated_salaries)

    # Floor the calculated salary using the relevant minimums
    return np.round(max(calculated_salaries), 2)

def SmoothNewContract():
    # Will want this function to calculate the new smoothed contract bases on year reamaining + years extended
    pass

In [15]:
# Calculate BaselineEYS for every player in the specified current season-week
ext_df = contracts_df[ contracts_df['season'] == ( current_season if (current_week > 0) else current_season-1 ) ][['player_id','player_name','conf','pos','salary','contract_years']]
ext_df['baselineEYS'] = ext_df.apply(lambda x: CalculateBaselineEYS(contracts_df, x['player_id'], x['conf'], 2024, 0), axis=1)

# Update some data, if necessary
ext_df['salary'] = ext_df['salary'] * ( 1. if (current_week > 0) else 1.1 )
ext_df['contract_years'] = ext_df['contract_years'] - ( 0. if (current_week > 0) else 1 )
ext_df['current_season'] = current_season
ext_df['current_week'] = current_week

# Save a copy of the BaselineEYS df
ext_df.to_csv(f'{save_label}_BaselineEYS.csv', index=False)

In [16]:
ext_df

,player_id,player_name,conf,pos,salary,contract_years,baselineEYS,current_season,current_week
431,12742,"Howard, Xavien",AFC,CB,6.391,0.0,6.39,2024,0
432,12742,"Howard, Xavien",NFC,CB,5.753,0.0,5.75,2024,0
433,13470,"Moore, Kenny",AFC,CB,5.467,1.0,5.47,2024,0
434,13710,"Davis, Carlton",AFC,CB,4.928,0.0,4.93,2024,0
435,12711,"Ramsey, Jalen",AFC,CB,4.829,1.0,6.07,2024,0
...,...,...,...,...,...,...,...,...,...
5978,16199,"Palmer, Trey",AFC,WR,0.891,2.0,3.64,2024,0
5979,16201,"Hutchinson, Xavier",AFC,WR,0.891,2.0,3.64,2024,0
5980,16207,"Iosivas, Andrei",NFC,WR,0.891,2.0,3.64,2024,0
5981,16299,"Davis, Derius",AFC,WR,0.891,2.0,3.64,2024,0
